# Model Testing 


In [2]:
# import the stuff we need
import langchain
import ollama

## model selections 
 - Qwen 2.5 coder (0.5b) 
 - Qwen 2.5 coder (32b) 


In [ ]:
# pick what model to use 
model1 = 'qwen2.5-coder:0.5b'
model2 = 'qwen2.5-coder:32b'


## Prompt templates
 - one shot 
 - multi shot
 - chain of thought 
---

In [6]:
from langchain.prompts import PromptTemplate

oneShot = PromptTemplate(
    input_variables=["method_code"],
    template='''
Write a clear and professional method header in Javadoc style. I will provide some examples.

method_code: 
public int factorial(int n) {{
    if (n < 0) {{
        throw new IllegalArgumentException("Input must be non-negative.");
    }}
    int result = 1;
    for (int i = 2; i <= n; i++) {{
        result *= i;
    }}
    return result;
}}

good output:
/**
 * Calculates the factorial of a given non-negative integer.
 *
 * @param n the number to compute the factorial of
 * @return the factorial of n
 * @throws IllegalArgumentException if n is negative
 */

now given {method_code}:

write a Javadoc header for the method above.

'''
)


## Sample code 


In [8]:
# add some content 
shortMethod = oneShot.format(
    method_code = """
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         """)

In [9]:
longMethodWithHelper = oneShot.format(method_code="""\
private void rehash() {{

    int newCapacity = Primes.nextPrime(table.length * 2);
    HexPiece[] oldTable = table;
    table = new HexPiece[newCapacity];
    size = 0;

    for (HexPiece head : oldTable) {{
        if (head == null) continue;
        HexPiece current = head;
        do {{
            HexPiece next = current.nextInChain;
            int newIndex = locate(current.location);

            if (table[newIndex] == null) {{
                current.nextInChain = current;
                table[newIndex] = current;
            }} else {{
                current.nextInChain = table[newIndex].nextInChain;
                table[newIndex].nextInChain = current;
            }}
            size++;
            current = next;
        }} while (current != head);
    }}
}}

public boolean add(HexPiece p) {{
    assert wellFormed() : "Invariant broken before add";

    if (p == null || p.location == null || p.terrain == null)
        throw new NullPointerException("Piece, location, and terrain must not be null");

    int index = locate(p.location);
    HexPiece head = table[index];

    if (head != null) {{
        HexPiece current = head;
        do {{
            if (current.location.equals(p.location)) {{
                current.terrain = p.terrain;
                version++;
                assert wellFormed() : "Invariant broken after add (update)";
                return false;
            }}
            current = current.nextInChain;
        }} while (current != head);

        p.nextInChain = head.nextInChain;
        head.nextInChain = p;
    }} else {{
        p.nextInChain = p;
        table[index] = p;
    }}

    connect(p);
    size++;
    version++;

    if (size >= table.length) {{
        rehash();
    }}

    assert wellFormed() : "Invariant broken after add (insert)";
    return true;
}}
""")


## Evaluating outputs

In [11]:
# Run model
from ollama import chat

response = chat(model=model1, messages=[{'role': 'user', 'content': shortMethod}])
print(response['message']['content'])


/**
 * Connects a HexPiece to its neighbors based on given directions.
 *
 * @param piece The HexPiece to connect to its neighbors.
 */
public void connect(HexPiece piece) {
    HexCoordinate h = piece.getLocation();
    for (HexDirection d : HexDirection.values()) {
        HexCoordinate h2 = d.move(h);
        HexPiece p2 = findPiece(h2);
        if (p2 != null) {
            p.neighbors[d.ordinal()] = p2;
            p2.neighbors[d.reverse().ordinal()] = p;
        }
    }
}


In [48]:
# Run model
response = chat(model=model1, messages=[{'role': 'user', 'content': longMethodWithHelper}])
print(response['message']['content'])


```java
/**
 * Adds a new piece to the terrain of an existing piece.
 *
 * @param p The piece to be added
 * @return True if the operation is successful, false otherwise
 */
public boolean add(HexPiece p) {
    assert wellFormed() : "Invariant broken before add";

    if (p == null || p.location == null || p.terrain == null)
        throw new NullPointerException("Piece, location, and terrain must not be null");

    int index = locate(p.location);
    HexPiece head = table[index];

    if (head != null) {{
        HexPiece current = head;
        do {{
            if (current.location.equals(p.location)) {{
                current.terrain = p.terrain;
                version++;
                assert wellFormed() : "Invariant broken after add (update)";
                return false;
            }}
            current = current.nextInChain;
        }} while (current != head);

        p.nextInChain = head.nextInChain;
        head.nextInChain = p;
    }} else {{
        p.nextInChain =